# pandas

References
- [Pandas API](https://pandas.pydata.org/docs/reference/)
- [Pandas Lexicon](https://github.com/anitacheung/alpha/blob/ff84e550b94634b5243878a03457713c37a8ad8a/python/lexicon-pandas.csv)

In [7]:
import pandas as pd

## Pandas Introduction
- Wrapper for numpy (therefore maintains functionality of numpy)
- Methods: df.method()
- Functions: pd.function(df)

## Getting and filtering data
### Filtering by Date
Tags: <a name="Reading_CSV_data_into_Pandas">Reading CSV data into pandas</a>, <a name="Filtering_to_specific_dates">Filtering to specific dates</a>, <a name="Date_slicing">Date slicing</a>

- Filtering via joining data is faster than through booleans: [code](https://github.com/anitacheung/alpha/blob/9164592c1db3e050d0e720fd1aff5bf7f871272c/python/pandas-timing.py)

In [8]:
import pandas as pd
import datetime as dt

def get_data():                                     # Function assumed for later examples
    df = pd.read_csv('filename.csv')              
    df = date_filtering(df)

def date_filtering(df):
    # Specific day: subset of existing dataframe
    df_date_subset = df[df.Date == '01/01/2000']    # Method 1
    # df_date_subset = df[df['Date]=='01/01/2000']  # Method 2

    # Date range: from empty dataframe
    symbols = ['GOOG', 'IBM', 'GLD']
    start = '2000-01-01'
    end = '2000-02-01'
    dates= pd.date_range(start, end)                
    df_range = pd.DateFrame(index=dates)            # Column automatically named 'Date'
    for symbol in symbols:
        df_ticker = pd.read_csv('data/{}.csv'.format(symbol), 
                                index_col=dates, 
                                parse_dates=True,
                                usecols=['Date', 'Adj Close'],
                                na_values=['nan'])
        df_ticker = df_ticker.rename(columns={'Adj Close': symbol})
        df_range = df_range.join(df_ticker)         # Left join by default
    return df_range

def other_slicing(df):
    df.head()                   # Gets the first 5 elements
    df.tail(n)                  # Gets the last n elements

### Data Slicing
Tags: <a name="Symbol_slicing">Symbol slicing</a>

In [9]:
def symbol_slicing(df):
    df_subset = df[start_index: end_index, ['GOOG', 'GLD']]
    df_subset.ix[start_rowname:end_rowname_incl, start_colname:end_colname_incl]        # Deprecated
    df_subset.loc[start_rowname:end_rowname_incl, start_colname:end_colname_incl]       # same as .ix
    df_subset_iloc[start_rowindex:end_rowindex_notinc, [colnames]]

### Cleaning data
Tags: <a name="ffill_bfill">Forward and Backward Fill</a>

- Copy and then set values to 0 is faster than creating a new zero array; copy is an array of pointers until the values diverge

In [10]:
def clean_df(df):
    
    df.fillna(method='ffill', inplace=True)                  # forward fill na
    df.ffill()                                               # forward fill na
    df.fillna(method='bfill', inplace=True)                  # backfill na
    df.bfill()                                               # backfill na
    df.dropna(subset=['colname'], inplace=True)              # drop na
    df = df.rename(columns={'oldname':'newname'})            # rename columns
    df2 = df.copy(deep=True)                                 # copies df

## Operations

- Pandas will perform element wise based on index unless the underlying numpy array is pulled out

In [11]:
df[1:] + df[:-1].values
df * 2                      # multiples each element
df / 2                      # divides each element

NameError: name 'df' is not defined

## Statistics
Tags: <a name="Gross_statics_on_dataframes">Gross statistics on dataframes</a>, <a name="Correlation">Correlation</a>

In [ ]:
def stock_stats(df):
    max_close = df['Close'].max()               # max
    min_close = df['Close'].min()               # min
    mean_volume = df['Volume'].mean()           # mean
    median_volume = df['Volume'].median()       # median
    df['Close'].std()                           # std

def stock_relationships(df):
    df.corr(method='pearson')                   # correlation  

## Technical Indicators
Tags: <a name="Rolling_statistics_on_dataframes">Rolling statistics on dataframes</a>, <a name="Plotting_a_technical_indicator_(Bollinger_Bands)">Plotting a technical indicator (Bollinger Bands)</a>

In [ ]:
rm_df = pd.rolling_mean(df[ticker], window=num_days)            # 1. Rolling mean
rstd_df = pd.rolling_std(df[ticker], window=num_days)           # 2. Rolling std
uband_df = rm_df + (2 * rstd_df)                                # 3. Upper and lower bands
lband_df = rm_df - (2 * rstd_df)  
rmax_df = pd.rolling_max(df[ticker], window=num_days)           # Rolling max
rmin_df = pd.rolling_min(df[ticker], window=num_days)           # Rolling min   

NameError: name 'pd' is not defined

## Plotting Dataframes
Tags: <a name="Plotting">Plotting</a>, <a name="Normalizing">Normalizing</a>

- Plotting dataframes

In [ ]:
import matplotlib.pyplot as pyplot 

ax = df[ticker1].plot(title='title', label='ticker1')
df.plot(label='ticker2', ax=ax)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.legend(loc='upper left')
plt.show()

### Histograms
Tags: <a name="Histogram_of_daily_returns">Histogram of daily returns</a>

- Important Features: mean, standard deviation, kurtosis 

In [ ]:
df[ticker1].hist(bins=num_bins, label=ticker1)                      # Plot as histogram
df[ticker2].hist(bins=num_bins, label=ticker2)
plt.axvline(df.mean(), color='w), linestyle='dashed', linewidth=2)  # Plot mean line
plt.axvline(-df.std(), color='w), linestyle='dashed', linewidth=2)  # Plot std lines
plt.axvline(df.std(), color='w), linestyle='dashed', linewidth=2)
plt.text(x, y, df.kurtosis())                                       # Annotate with kurtosis value
plt.legend(loc='upper left')
plt.show()

### Scatter Plots
Tags: <a name="Scatter_plots">Scatter plots</a>

import numpy as np

def plot_data(df):
    df[['Adj Close', 'Close']].plot()  # Assumes date is already set as index
    plt.show()

def compare_stocks(df):
    df.plot(kind='scatter', x=ticker1, y=ticker2)
    beta, alpha = np.polyfit(df[ticker1], df[ticker2], 1)               # See numpy
    plt.plot(df[ticker1], beta * df[ticker1] + alpha, '-', color='r')   # ticker1 is on x
    plt.show()

## Merging data
Tags: <a name="Reading_data_for_multiple_stocks_into_the_structure">Reading data for multiple stocks into the structure</a>

## Data Manipulation

In [ ]:
df.shift(1)     # shifts index assignment down by one (leaves offset values as NaNs)